# Running Monte Carlo Transport Independently

This tutorial demonstrates how to run the Monte Carlo transport loop directly using `Simulation.from_config` without running full TARDIS iterations. This approach gives you direct control over the Monte Carlo transport process.

In [1]:
import os
from pathlib import Path

import astropy.units as u
import numpy as np

from tardis.simulation import Simulation
from tardis.io.configuration.config_reader import Configuration
from tardis.io.atom_data import AtomData
from tardis.transport.montecarlo.estimators.radfield_mc_estimators import (
    initialize_estimator_statistics,
)
from tardis.transport.montecarlo.montecarlo_main_loop import montecarlo_main_loop
from tardis.transport.montecarlo.packets.packet_trackers import (
    generate_rpacket_last_interaction_tracker_list,
    generate_rpacket_tracker_list,
)

/home/runner/work/tardis/tardis/tardis/__init__.py:17: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

Initializing tabulator and plotly panel extensions for widgets to work


In [2]:
!wget -q -nc https://raw.githubusercontent.com/tardis-sn/tardis/master/docs/tardis_example.yml

In [3]:
# User-configurable variables
CONFIG_FILE_NAME = "tardis_example.yml"
NUMBER_OF_PACKETS = 10000
NUMBER_OF_VPACKETS = 0  # Set to 0 to disable virtual packets
ITERATION_NUMBER = 1
SHOW_PROGRESS_BARS = True
TOTAL_ITERATIONS = 1
ENABLE_RPACKET_TRACKING = True  # True: full tracking, False: last interaction only

In [4]:
# Setup simulation state from config
config_file = Path(CONFIG_FILE_NAME)
if not config_file.exists():
    raise FileNotFoundError(f"Configuration file {CONFIG_FILE_NAME} not found")

config = Configuration.from_yaml(str(config_file))
atom_data = AtomData.from_hdf("kurucz_cd23_chianti_H_He_latest.h5")
sim = Simulation.from_config(config, atom_data=atom_data)

print("Simulation created successfully!")

Number of density points larger than number of shells. Assuming inner point irrelevant


model_isotope_time_0 is not set in the configuration. Isotopic mass fractions will not be decayed and is assumed to be correct for the time_explosion. THIS IS NOT RECOMMENDED!


Simulation created successfully!


In [5]:
# Initialize opacity and macro atom states manually
sim.opacity_state = sim.opacity.legacy_solve(sim.plasma)

if sim.macro_atom is not None:
    sim.macro_atom_state = sim.macro_atom.solve(
        sim.plasma.j_blues,
        sim.opacity_state.beta_sobolev,
        sim.plasma.stimulated_emission_factor,
    )
else:
    sim.macro_atom_state = None

print("Opacity and macro atom states initialized!")

Opacity and macro atom states initialized!


In [6]:
# Extract states from simulation
geometry_state = sim.simulation_state.geometry
opacity_state = sim.opacity_state
montecarlo_configuration = sim.transport.montecarlo_configuration
time_explosion = sim.simulation_state.time_explosion.to(u.s).value
spectrum_frequency_grid = sim.transport.spectrum_frequency_grid.to(u.Hz).value
packet_source = sim.transport.packet_source

# Initialize estimators
tau_sobolev_shape = opacity_state.tau_sobolev.shape
gamma_shape = (0, geometry_state.no_of_shells)
estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)

# Convert to numba-compatible versions
geometry_state_numba = geometry_state.to_numba()
line_interaction_type = montecarlo_configuration.LINE_INTERACTION_TYPE
opacity_state_numba = opacity_state.to_numba(sim.macro_atom_state, line_interaction_type)

print("Monte Carlo states prepared!")

Monte Carlo states prepared!


In [7]:
# Create packet collection
seed_offset = montecarlo_configuration.PACKET_SEEDS
packet_collection = packet_source.create_packets(NUMBER_OF_PACKETS, seed_offset)

# Setup packet tracking
if ENABLE_RPACKET_TRACKING:
    rpacket_trackers = generate_rpacket_tracker_list(
        NUMBER_OF_PACKETS,
        montecarlo_configuration.INITIAL_TRACKING_ARRAY_LENGTH,
    )
    print("Using full RPacket tracking")
else:
    rpacket_trackers = generate_rpacket_last_interaction_tracker_list(
        NUMBER_OF_PACKETS
    )
    print("Using last interaction tracking only")


Using full RPacket tracking


In [8]:
# Run the Monte Carlo main loop
v_packets_energy_hist, last_interaction_tracker, vpacket_tracker = (
    montecarlo_main_loop(
        packet_collection,
        geometry_state_numba,
        time_explosion,
        opacity_state_numba,
        montecarlo_configuration,
        estimators,
        spectrum_frequency_grid,
        rpacket_trackers,
        NUMBER_OF_VPACKETS,
        SHOW_PROGRESS_BARS
    )
)

print("Monte Carlo transport completed successfully!")

/home/runner/work/tardis/tardis/tardis/transport/montecarlo/montecarlo_main_loop.py:150: NumbaTypeSafetyWarning:

unsafe cast from uint64 to int64. Precision may be lost.



Monte Carlo transport completed successfully!


In [9]:
# Inspect results
print("Monte Carlo Results:")
print(f"V-packet energy histogram shape: {v_packets_energy_hist.shape}")
print(f"Last interaction tracker type: {type(last_interaction_tracker)}")
if ENABLE_RPACKET_TRACKING:
    print(f"V-packet tracker available: {type(vpacket_tracker)}")

# Results are now available in the notebook for further analysis:
# - v_packets_energy_hist: energy histogram of virtual packets
# - last_interaction_tracker: final interaction data for all packets  
# - vpacket_tracker: virtual packet collection (if enabled)
# - sim: original simulation object

Monte Carlo Results:
V-packet energy histogram shape: (10001,)
Last interaction tracker type: <class 'numba.experimental.jitclass.boxing.LastInteractionTracker'>
V-packet tracker available: <class 'numba.experimental.jitclass.boxing.VPacketCollection'>


## Usage Instructions

### Configuration Options:
- **NUMBER_OF_PACKETS**: Number of packets to simulate (default: 10000)
- **NUMBER_OF_VPACKETS**: Number of virtual packets per interaction (0 = disabled)
- **ENABLE_RPACKET_TRACKING**: True for full tracking, False for last interaction only

### For Development and Debugging:
1. Set `NUMBA_DISABLE_JIT=1` in your environment variables to disable Numba JIT compilation
2. Set breakpoints in cell 8 (the Monte Carlo main loop call) 
3. Run the notebook in debug mode to step through the transport process

### Key Advantages Over Full TARDIS Run:
- **Direct control**: Access Monte Carlo transport without full simulation iterations
- **Manual state initialization**: All states explicitly prepared for transparency
- **Clean separation**: Setup and execution are clearly separated
- **Educational**: Perfect for understanding Monte Carlo transport physics step-by-step

# Running Monte Carlo Transport Loop

This tutorial demonstrates how to run the `montecarlo_main_loop` function directly with the TARDIS quickstart configuration.

In [10]:
import numpy as np
import astropy.units as u
from pathlib import Path

from tardis import run_tardis
from tardis.transport.montecarlo.montecarlo_main_loop import montecarlo_main_loop
from tardis.transport.montecarlo.packets.packet_collections import PacketCollection
from tardis.transport.montecarlo.packets.packet_trackers import (
    generate_rpacket_tracker_list,
    generate_rpacket_last_interaction_tracker_list
)

In [11]:
print("Monte Carlo Results:")
print(f"V-packet energy histogram shape: {v_packets_energy_hist.shape}")
print(f"Last interaction tracker type: {type(last_interaction_tracker)}")
if ENABLE_RPACKET_TRACKING:
    print(f"V-packet tracker available: {type(vpacket_tracker)}")

Monte Carlo Results:
V-packet energy histogram shape: (10001,)
Last interaction tracker type: <class 'numba.experimental.jitclass.boxing.LastInteractionTracker'>
V-packet tracker available: <class 'numba.experimental.jitclass.boxing.VPacketCollection'>


In [12]:
sim = run_tardis(CONFIG_FILE_NAME, show_progress_bars=SHOW_PROGRESS_BARS)

Auto-detected Sphinx build environment


Auto-detected Sphinx build environment


Embedding the final state for Jupyter environments


In [13]:
from tardis.transport.montecarlo.estimators.radfield_mc_estimators import initialize_estimator_statistics

# Get the geometry from simulation_state
geometry_state = sim.simulation_state.geometry
# Get the opacity state from the simulation (not from transport.transport_state)
opacity_state = sim.opacity_state
montecarlo_configuration = sim.transport.montecarlo_configuration
time_explosion = sim.simulation_state.time_explosion.to(u.s).value

# Initialize estimators using the same function used in TARDIS
tau_sobolev_shape = opacity_state.tau_sobolev.shape
# For continuum estimators, check if we have continuum processes
if hasattr(opacity_state, 'continuum_state') and opacity_state.continuum_state is not None:
    gamma_shape = (opacity_state.continuum_state.photo_ion_idx.max() + 1, geometry_state.no_of_shells)
else:
    gamma_shape = (0, geometry_state.no_of_shells)

estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)
spectrum_frequency_grid = sim.transport.spectrum_frequency_grid.to(u.Hz).value
packet_source = sim.transport.packet_source

# Convert objects to numba-compatible versions
geometry_state_numba = geometry_state.to_numba()
macro_atom_state = sim.macro_atom_state if hasattr(sim, 'macro_atom_state') else None
line_interaction_type = montecarlo_configuration.LINE_INTERACTION_TYPE
opacity_state_numba = opacity_state.to_numba(macro_atom_state, line_interaction_type)

In [14]:
seed_offset = montecarlo_configuration.PACKET_SEEDS
packet_collection = packet_source.create_packets(NUMBER_OF_PACKETS, seed_offset)

if ENABLE_RPACKET_TRACKING:
    rpacket_trackers = generate_rpacket_tracker_list(
        NUMBER_OF_PACKETS,
        montecarlo_configuration.INITIAL_TRACKING_ARRAY_LENGTH,
    )
else:
    rpacket_trackers = generate_rpacket_last_interaction_tracker_list(
        NUMBER_OF_PACKETS
    )

In [15]:
%%timeit
# Run the Monte Carlo main loop
# Reinitialize estimators (they get modified by the main loop)
estimators = initialize_estimator_statistics(tau_sobolev_shape, gamma_shape)

# Run the Monte Carlo main loop
v_packets_energy_hist, last_interaction_tracker, vpacket_tracker = montecarlo_main_loop(
    packet_collection,
    geometry_state_numba,
    time_explosion,
    opacity_state_numba,
    montecarlo_configuration,
    estimators,
    spectrum_frequency_grid,
    rpacket_trackers,
    NUMBER_OF_VPACKETS,
    SHOW_PROGRESS_BARS
)



166 ms ± 4.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
